<a href="https://colab.research.google.com/github/phubestp/thailand-speedtest-data-analysis/blob/main/Thailand_Speedtest_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thailand Speedtest Analysis



In [ ]:
!pip install geopy

In [ ]:
!pip install -q kaleido

In [ ]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
!wget -q https://github.com/google/fonts/raw/main/ofl/sarabun/Sarabun-Regular.ttf

### Library

In [ ]:
import os
import db_dtypes
import dask.dataframe as dd
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dask.diagnostics import ProgressBar
from tqdm import tqdm
import seaborn as sns
import matplotlib
import matplotlib.ticker as ticker
from geopy.distance import geodesic
import random
import pytz
import geopandas as gpd
import json
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from google.colab import output
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

In [ ]:
%reload_ext autoreload
%autoreload 2
output.enable_custom_widget_manager()

In [ ]:
matplotlib.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
matplotlib.rc('font', family='Sarabun')

### เตรียมข้อมูลการทดสอบ

In [ ]:
province_region = {
  "North": [
    "Chiang Rai", "Nan", "Phayao", "Chiang Mai", "Mae Hong Son",
    "Phrae", "Lampang", "Lamphun", "Uttaradit"
  ],
  "Central": [
    "Bangkok", "Phitsanulok", "Sukhothai", "Phetchabun", "Phichit",
    "Kamphaeng Phet", "Nakhon Sawan", "Lopburi", "Chai Nat",
    "Uthai Thani", "Sing Buri", "Ang Thong", "Saraburi",
    "Phra Nakhon Si Ayutthaya", "Suphan Buri", "Nakhon Nayok",
    "Pathum Thani", "Nonthaburi", "Nakhon Pathom",
    "Samut Prakan", "Samut Sakhon", "Samut Songkhram"
  ],
  "Northeast": [
    "Nong Khai", "Nakhon Phanom", "Sakon Nakhon", "Udon Thani",
    "Nong Bua Lam Phu", "Loei", "Mukdahan", "Kalasin", "Khon Kaen",
    "Amnat Charoen", "Yasothon", "Roi Et", "Maha Sarakham", "Chaiyaphum",
    "Nakhon Ratchasima", "Buriram", "Surin", "Si Sa Ket", "Ubon Ratchathani"
  ],
  "East": [
    "Sa Kaeo", "Prachin Buri", "Chachoengsao", "Chon Buri",
    "Rayong", "Chanthaburi", "Trat"
  ],
  "West": [
    "Tak", "Kanchanaburi", "Ratchaburi", "Phetchaburi",
    "Prachuap Khiri Khan"
  ],
  "South": [
    "Chumphon", "Ranong", "Surat Thani", "Nakhon Si Thammarat",
    "Krabi", "Phang Nga", "Phuket", "Phatthalung", "Trang",
    "Pattani", "Songkhla", "Satun", "Narathiwat", "Yala"
  ]
}
province_to_region = {
    province: region
    for region, provinces in province_region.items()
    for province in provinces
}

In [ ]:
result_df = dd.read_parquet("/content/results_2023_2025.parquet", blocksize="64MB")

In [ ]:
result_ddf = result_df.copy()
result_ddf.compute().shape

In [ ]:
result_ddf = result_ddf.map_partitions(lambda df: df.reset_index(drop=True))
result_ddf.head()

เปลี่ยนจากเมืองเป็นจังหวัด

In [ ]:
with open('/content/city_to_province.json') as json_data:
    non_province_mapping = json.load(json_data)
result_ddf["city"] = result_ddf["city"].str.strip().str.title()
result_ddf = result_ddf.dropna(subset=["city"])
result_ddf["city"] = result_ddf["city"].map(
    non_province_mapping, meta=('city', 'string')
).fillna(result_ddf["city"])
print(result_ddf["city"].compute().unique())

In [ ]:
print(result_ddf[result_ddf["mean_throughput_mbps"] == 0].compute().shape)
result_ddf = result_ddf[result_ddf["mean_throughput_mbps"] > 0]

In [ ]:
print(result_ddf[result_ddf["type"] == "download"].compute().shape[0])
print(result_ddf[result_ddf["type"] == "upload"].compute().shape[0])

In [ ]:
client_df = dd.read_parquet("/content/client_complete.parquet")
client_ddf = client_df.map_partitions(lambda df: df.reset_index(drop=True))

In [ ]:
client_ddf.dtypes

In [ ]:
client_ddf = client_ddf.drop(columns=["mobile", "category", "network_type", "description"])
isp_info = pd.read_csv("/content/isp_complete.csv")
client_ddf = client_ddf.merge(isp_info, on="isp", how="left")
client_ddf.dtypes

In [ ]:
client_ddf.head()

In [ ]:
result_mapping_df = dd.merge(result_ddf, client_ddf, on="client_ip", how="left")
result_mapping_df = result_mapping_df.drop(columns=["date", "congestion_control"])
result_mapping_df.dtypes

In [ ]:
result_mapping_df.head()

In [ ]:
result_mapping_df.dtypes

In [ ]:
server_info = pd.read_csv("/content/server_info.csv")

def is_finite(value):
    return np.isfinite(value) and value != 0.0

def get_distance(row):
    client_x = row['latitude']
    client_y = row['longitude']

    if not is_finite(client_x) or not is_finite(client_y):
        return np.nan

    server_data = server_info.loc[server_info['server_ip'] == row['server_ip']]
    if server_data.empty:
        return np.nan

    server_x = server_data['latitude'].values[0]
    server_y = server_data['longitude'].values[0]

    if not is_finite(server_x) or not is_finite(server_y):
        return np.nan

    try:
        distance = geodesic((client_x, client_y), (server_x, server_y)).kilometers
    except ValueError:
        distance = np.nan

    return distance

result_mapping_df['distance_from_server'] = result_mapping_df.apply(
    get_distance, axis=1, meta=('distance_from_server', 'float64')
)

In [ ]:
result_mapping_df.head()

In [ ]:
result_mapping_df = result_mapping_df.drop(columns=[
    "id"
])

In [ ]:
result_mapping_df["hour_utc"] = result_mapping_df["test_time"].dt.hour
result_mapping_df["hour_th"] = (result_mapping_df["hour_utc"] + 7) % 24
result_mapping_df["year"] = result_mapping_df["test_time"].dt.year
result_mapping_df["month"] = result_mapping_df["test_time"].dt.month

In [ ]:
result_mapping_df['final_duration'] = result_mapping_df['speedtest_time'].fillna(result_mapping_df['duration'])

In [ ]:
result_mapping_df.head(1)

In [ ]:
result_mapping_df = result_mapping_df.drop(columns=['speedtest_time', 'duration'])
result_mapping_df = result_mapping_df.rename(columns={'final_duration': 'duration'})

In [ ]:
result_mapping_df = result_mapping_df.rename(columns={
    "hour_th": "hour",
})
result_mapping_df['city'] = result_mapping_df['city'].astype('category')
result_mapping_df['isp'] = result_mapping_df['isp'].astype('category')
result_mapping_df['type'] = result_mapping_df['type'].astype('category')
result_mapping_df['category'] = result_mapping_df['category'].astype('category')

In [ ]:
result_mapping_df = result_mapping_df.drop(columns=[
    "mobile", "hour_utc"
])

In [ ]:
result_mapping_df.dtypes

In [ ]:
def remove_outliers_iqr(df, group_cols, target_col):
    def _remove(group):
        q1 = group[target_col].quantile(0.25)
        q3 = group[target_col].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        return group[(group[target_col] >= lower) & (group[target_col] <= upper)]

    return df.groupby(group_cols, group_keys=False).apply(_remove)

In [ ]:
grouped_counts = result_mapping_df.groupby(["city", "network_type", "type"]).size().reset_index().compute()

In [ ]:
valid_keys = set(tuple(x) for x in grouped_counts[grouped_counts[0] >= 30][["city", "network_type", "type"]].values)
print(valid_keys)
def filter_valid_groups(df, valid_keys):
    return df[df[["city", "network_type", "type"]].apply(tuple, axis=1).isin(valid_keys)]

filtered_result_mapping_df = result_mapping_df.map_partitions(filter_valid_groups, valid_keys)

### Confidence Interval

In [ ]:
ci_df = filtered_result_mapping_df[["city", "network_type", "type", "mean_throughput_mbps"]].dropna().compute()
ci_df["log_throughput"] = np.log(ci_df["mean_throughput_mbps"] + 1)

In [ ]:
from scipy.stats import norm

grouped = ci_df.groupby(["city", "network_type", "type"])["log_throughput"]
summary = grouped.agg(["mean", "std", "count"]).reset_index()
z = norm.ppf(0.95)
summary["margin_of_error"] = z * summary["std"] / np.sqrt(summary["count"])
summary["ci_lower"] = summary["mean"] - summary["margin_of_error"]
summary["ci_upper"] = summary["mean"] + summary["margin_of_error"]

In [ ]:
summary["ci_width"] = summary["ci_upper"] - summary["ci_lower"]
summary.sort_values("ci_width")

### ดูตาม Latitude & Longtitude

In [ ]:
lat_lon_df = result_mapping_df[["city", "latitude", "longitude", "network_type", "type", "mean_throughput_mbps"]].compute()

In [ ]:
lat_lon_summary_df = lat_lon_df.groupby(
    ["city", "latitude", "longitude", "network_type", "type"]
).agg(
    mean_throughput_mbps=('mean_throughput_mbps', 'mean'),
    count=('mean_throughput_mbps', 'count'),
).dropna().reset_index()

In [ ]:
lat_lon_summary_df

### Correlation

In [ ]:
columns_needed = [
    "mean_throughput_mbps", "min_rtt", "loss_rate",
    "distance_from_server", "duration"
]

df_sampled = result_mapping_df[columns_needed].sample(frac=0.03).compute()
corr_matrix = df_sampled.corr()

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", center=0)
plt.title("Correlation Matrix")
plt.show()

### การกระจายความเร็วในแต่ละจังหวัด

In [ ]:
dis_df = result_mapping_df[["city", "network_type", "type", "mean_throughput_mbps"]].compute()
dis_df

#### Broadband

In [ ]:
broadband_df = dis_df[dis_df["network_type"] == "broadband"]

broadband_download_df = broadband_df[broadband_df["type"] == "download"]
broadband_upload_df = broadband_df[broadband_df["type"] == "upload"]

common_cities = set(broadband_download_df['city']).intersection(broadband_upload_df['city'])

broadband_download_df = broadband_download_df[broadband_download_df['city'].isin(common_cities)]
broadband_upload_df = broadband_upload_df[broadband_upload_df['city'].isin(common_cities)]

city_counts = (
    broadband_download_df['city']
    .value_counts()
    .sort_values(ascending=False)
)

city_labels = [f"{city} ({count})" for city, count in city_counts.items()]

fig, axes = plt.subplots(2, 1, figsize=(26, 12))

sns.boxplot(
    data=broadband_download_df,
    x='city',
    y='mean_throughput_mbps',
    order=city_counts.index,
    ax=axes[0],
    flierprops=dict(marker='o', color='gray', markersize=5, alpha=0.25),
    palette=["skyblue"],
)
axes[0].set_title('Download Speed (Broadband)')
axes[0].set_xlabel('Province (with number of tests)')
axes[0].set_ylabel('Speed (Mbps)')
axes[0].set_xticks(range(len(city_labels)))
axes[0].set_xticklabels(city_labels, rotation=90)

sns.boxplot(
    data=broadband_upload_df,
    x='city',
    y='mean_throughput_mbps',
    order=city_counts.index,
    ax=axes[1],
    palette=["salmon"],
    flierprops=dict(marker='o', color='gray', markersize=5, alpha=0.25),
)
axes[1].set_title('Upload Speed (Broadband)')
axes[1].set_xlabel('Province (with number of tests)')
axes[1].set_ylabel('Speed (Mbps)')
axes[1].set_xticks(range(len(city_labels)))
axes[1].set_xticklabels(city_labels, rotation=90)

plt.tight_layout()
plt.show()

#### Cellular

In [ ]:
cellular_df = dis_df[dis_df["network_type"] == "cellular"]

cellular_download_df = cellular_df[cellular_df["type"] == "download"]
cellular_upload_df = cellular_df[cellular_df["type"] == "upload"]

common_cities = set(cellular_download_df['city']).intersection(cellular_upload_df['city'])

cellular_download_df = cellular_download_df[cellular_download_df['city'].isin(common_cities)]
cellular_upload_df = cellular_upload_df[cellular_upload_df['city'].isin(common_cities)]

city_counts = (
    cellular_download_df['city']
    .value_counts()
    .sort_values(ascending=False)
)

city_labels = [f"{city} ({count})" for city, count in city_counts.items()]

fig, axes = plt.subplots(2, 1, figsize=(26, 12))

sns.boxplot(
    data=cellular_download_df,
    x='city',
    y='mean_throughput_mbps',
    order=city_counts.index,
    ax=axes[0],
    flierprops=dict(marker='o', color='gray', markersize=5, alpha=0.25),
    palette=["skyblue"],
    showfliers=False
)
axes[0].set_title('Download Speed (Cellular)')
axes[0].set_xlabel('Province (with number of tests)')
axes[0].set_ylabel('Speed (Mbps)')
axes[0].set_xticks(range(len(city_labels)))
axes[0].set_xticklabels(city_labels, rotation=90)

sns.boxplot(
    data=cellular_upload_df,
    x='city',
    y='mean_throughput_mbps',
    order=city_counts.index,
    ax=axes[1],
    palette=["salmon"],
    flierprops=dict(marker='o', color='gray', markersize=5, alpha=0.25),
    showfliers=False
)
axes[1].set_title('Upload Speed (Cellular)')
axes[1].set_xlabel('Province (with number of tests)')
axes[1].set_ylabel('Speed (Mbps)')
axes[1].set_xticks(range(len(city_labels)))
axes[1].set_xticklabels(city_labels, rotation=90)

for label in axes[0].get_xticklabels():
    text = label.get_text()
    city_name = text.split(' (')[0]
    count = int(text.split('(')[1].replace(')', ''))
    if count == 0:
        label.set_color('red')

for label in axes[1].get_xticklabels():
    text = label.get_text()
    city_name = text.split(' (')[0]
    count = int(text.split('(')[1].replace(')', ''))
    if count == 0:
        label.set_color('red')

plt.tight_layout()
plt.show()

### จังหวัดที่ความเร็วเฉลี่ยสูงที่สุดและต่ำที่สุด

In [ ]:
result_mapping_agg_df = result_mapping_df.groupby(
    ['city', 'type', 'network_type']
).agg(
    median_throughput_mbps=('mean_throughput_mbps', 'median'),
    count=('mean_throughput_mbps', 'count'),
).compute().dropna().reset_index()

In [ ]:
result_mapping_agg_df = result_mapping_agg_df[result_mapping_agg_df["count"] >= 30]

In [ ]:
top_fastest_by_group = result_mapping_agg_df.groupby(['type', 'network_type']).apply(
    lambda x: x.sort_values(by='median_throughput_mbps', ascending=False).head(10)
).reset_index(drop=True)

top_slowest_by_group = result_mapping_agg_df.groupby(['type', 'network_type']).apply(
    lambda x: x.sort_values(by='median_throughput_mbps', ascending=True).head(10)
).reset_index(drop=True)

In [ ]:
result_mapping_agg_df

#### เร็วที่สุด

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(12, 18))

types = ['download', 'download', 'upload', 'upload']
network_types = ['broadband', 'cellular', 'broadband', 'cellular']
titles = [
  "Top 10 Provinces with Fastest Download Speeds (Broadband)",
  "Top 10 Provinces with Fastest Download Speeds (Cellular)",
  "Top 10 Provinces with Fastest Upload Speeds (Broadband)",
  "Top 10 Provinces with Fastest Upload Speeds (Cellular)",
]

type_colors = {
    'download': 'skyblue',
    'upload': 'salmon'
}

for i, (t, c, title) in enumerate(zip(types, network_types, titles)):
    subset = top_fastest_by_group[
        (top_fastest_by_group['type'] == t) &
        (top_fastest_by_group['network_type'] == c)
    ]

    bars = axes[i].barh(
        subset['city'],
        subset['median_throughput_mbps'],
        color=type_colors[t]
    )
    axes[i].bar_label(bars, fmt='%.1f', padding=3)
    axes[i].set_xlabel('Median Throughput (Mbps)')
    axes[i].set_title(title)
    axes[i].invert_yaxis()

plt.tight_layout()
plt.show()

#### ช้าที่สุด

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(12, 18))

types = ['download', 'download', 'upload', 'upload']
network_types = ['broadband', 'cellular', 'broadband', 'cellular']
titles = [
  "Top 10 Provinces with Slowest Download Speeds (Broadband)",
  "Top 10 Provinces with Slowest Download Speeds (Cellular)",
  "Top 10 Provinces with Slowest Upload Speeds (Broadband)",
  "Top 10 Provinces with Slowest Upload Speeds (Cellular)",
]

type_colors = {
    'download': 'skyblue',
    'upload': 'salmon'
}

for i, (t, c, title) in enumerate(zip(types, network_types, titles)):
    subset = top_slowest_by_group[
        (top_slowest_by_group['type'] == t) &
        (top_slowest_by_group['network_type'] == c)
    ]

    bars = axes[i].barh(
        subset['city'],
        subset['median_throughput_mbps'],
        color=type_colors[t]
    )
    axes[i].bar_label(bars, fmt='%.1f', padding=3)
    axes[i].set_xlabel('Median Throughput (Mbps)')
    axes[i].set_title(title)
    axes[i].invert_yaxis()

plt.tight_layout()
plt.show()

### ดูการเปลี่ยนแปลงแต่ละเดือนรายจังหวัด

In [ ]:
df = result_mapping_df[["city", "year", "month", "type", "network_type", "mean_throughput_mbps"]].compute()

df = remove_outliers_iqr(df, ["city", "year", "month", "type", "network_type"], "mean_throughput_mbps")

monthly_city_df = df.groupby(
    ['city', 'year', 'month', 'type', 'network_type']
).agg(
    mean_throughput_mbps=('mean_throughput_mbps', 'mean'),
    count=('mean_throughput_mbps', 'count')
).reset_index()


In [ ]:
monthly_city_df = monthly_city_df.reset_index()

monthly_city_df['period'] = pd.to_datetime(
    monthly_city_df['year'].astype(str) + '-' +
    monthly_city_df['month'].astype(str).str.zfill(2) + '-01'
)

In [ ]:
monthly_city_df["region"] = monthly_city_df["city"].map(province_region)

In [ ]:
monthly_city_df["province_normalized"] = monthly_city_df["city"].str.strip().str.title()
monthly_city_df["region"] = monthly_city_df["province_normalized"].map(province_to_region)

In [ ]:
start_date = pd.to_datetime('2023-12-01')
end_date = pd.to_datetime('2025-03-01')

monthly_city_df["province_normalized"] = monthly_city_df["city"].str.strip().str.title()
province_to_region = {
    province: region
    for region, provinces in province_region.items()
    for province in provinces
}
monthly_city_df["region"] = monthly_city_df["province_normalized"].map(province_to_region)

filtered_df = monthly_city_df[
    (monthly_city_df['period'] >= start_date) &
    (monthly_city_df['period'] <= end_date) &
    (~monthly_city_df['mean_throughput_mbps'].isna())
].copy()

global_y_min = filtered_df['mean_throughput_mbps'].min() * 0.95
global_y_max = filtered_df['mean_throughput_mbps'].max() * 1.05

network_types = filtered_df['network_type'].dropna().unique()

for net_type in sorted(network_types):
    df_network = filtered_df[filtered_df['network_type'] == net_type]
    regions = df_network['region'].dropna().unique()

    for region in sorted(regions):
        df_region = df_network[df_network['region'] == region]
        provinces = sorted(df_region['province_normalized'].unique())
        chunk_size = 5

        for i in range(0, len(provinces), chunk_size):
            subset = provinces[i:i + chunk_size]
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

            dl_avg = (
                df_region[df_region['type'] == 'download']
                .groupby('period')['mean_throughput_mbps']
                .mean().reset_index()
            )
            ul_avg = (
                df_region[df_region['type'] == 'upload']
                .groupby('period')['mean_throughput_mbps']
                .mean().reset_index()
            )

            colors = sns.color_palette("Set2", len(subset))

            for idx, province in enumerate(subset):
                df_p = df_region[df_region['province_normalized'] == province].sort_values('period')
                ax1.plot(
                    df_p[df_p['type'] == 'download']['period'],
                    df_p[df_p['type'] == 'download']['mean_throughput_mbps'],
                    marker='o',
                    linewidth=1.5,
                    label=province,
                    color=colors[idx]
                )
                ax2.plot(
                    df_p[df_p['type'] == 'upload']['period'],
                    df_p[df_p['type'] == 'upload']['mean_throughput_mbps'],
                    marker='s',
                    linestyle='--',
                    linewidth=1.5,
                    label=province,
                    color=colors[idx]
                )

            ax1.plot(dl_avg['period'], dl_avg['mean_throughput_mbps'], color='black', linestyle=':', linewidth=2, label='Regional Download Speed Avg')
            ax2.plot(ul_avg['period'], ul_avg['mean_throughput_mbps'], color='black', linestyle=':', linewidth=2, label='Regional Upload Speed Avg')

            for ax, title in zip((ax1, ax2), ['Download Speed', 'Upload Speed']):
                ax.set_title(title)
                ax.set_xlabel('Month')
                ax.set_ylabel('Mean Speed (Mbps)')
                ax.set_xlim([start_date, end_date])
                ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
                ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
                ax.tick_params(axis='x', rotation=45)
                ax.grid(True)

            ax1.legend(title='Province', bbox_to_anchor=(1.05, 1), loc='upper left')
            ax2.legend(title='Province', bbox_to_anchor=(1.05, 1), loc='upper left')

            start_num = i + 1
            end_num = i + len(subset)

            if start_num == end_num:
                province_range = f'Province {start_num}'
            else:
                province_range = f'Provinces {start_num}-{end_num}'
            ax1.set_ylim(global_y_min, global_y_max)
            ax2.set_ylim(global_y_min, global_y_max)
            plt.suptitle(f'{region} Region - {net_type.capitalize()} - {province_range}', fontsize=16)
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            plt.show()

### จำนวนการทดสอบของแต่ละจังหวัด

In [ ]:
test_count = result_mapping_df.groupby("city").size().compute().sort_values()

plt.figure(figsize=(12, 12))
bars = plt.barh(test_count.index, test_count.values, color="royalblue")
plt.xscale("log")

tick_positions = [10, 100, 1_000, 10_000, 100_000, 1_000_000]
plt.xticks(tick_positions, [f"{int(tick):,}" for tick in tick_positions], fontsize=12)

for bar, value in zip(bars, test_count.values):
    text_offset = value * 0.05
    plt.text(
        value + text_offset,
        bar.get_y() + bar.get_height() / 2,
        f"{int(value):,}",
        va="center", ha="left", fontsize=8, color="black"
    )

plt.title("Test Count by Province", fontsize=16)
plt.xlabel("Test Count", fontsize=14)
plt.ylabel("Province", fontsize=12)
plt.yticks(fontsize=8)
plt.grid(axis="x", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

### จำนวนไอพีผู้ทดสอบในแต่ละจังหวัด

In [ ]:
unique_client_count = result_mapping_df.groupby("city")["client_ip"].nunique().compute()

In [ ]:
print(unique_client_count.sum())

In [ ]:
unique_client_count = unique_client_count.sort_values(ascending=True)

plt.figure(figsize=(12, 12))
x = np.arange(len(unique_client_count.index))

bars = plt.barh(x, unique_client_count.values, color="royalblue")

plt.xscale("log")

tick_positions = [10, 100, 1_000, 10_000, 100_000, 1_000_000]
plt.xticks(tick_positions, [f"{int(tick):,}" for tick in tick_positions], fontsize=12)

for bar, value in zip(bars, unique_client_count.values):
    text_offset = value * 0.05
    plt.text(
        value + text_offset,
        bar.get_y() + bar.get_height() / 2,
        f"{int(value):,}",
        va="center", ha="left", fontsize=8, color="black"
    )

plt.title("Unique Client IP Count by Province", fontsize=16)
plt.xlabel("Unique Client IP Count", fontsize=14)
plt.ylabel("Province", fontsize=12)

plt.yticks(x, unique_client_count.index, fontsize=8)
plt.grid(axis="x", linestyle="--", alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
unique_client_count = result_mapping_df.groupby("city")["client_ip"].nunique().compute().reset_index(name="count")
unique_client_count = unique_client_count.sort_values(by="count", ascending=False).head(10)

plt.figure(figsize=(12, 8))
x = np.arange(len(unique_client_count.index))

bars = plt.barh(x, unique_client_count["count"].values, color="royalblue")

plt.xscale("log")

tick_positions = [10, 100, 1_000, 10_000, 100_000, 1_000_000]
plt.xticks(tick_positions, [f"{int(tick):,}" for tick in tick_positions], fontsize=12)

for bar, value in zip(bars, unique_client_count["count"].values):
    text_offset = value * 0.05
    plt.text(
        value + text_offset,
        bar.get_y() + bar.get_height() / 2,
        f"{int(value):,}",
        va="center", ha="left", fontsize=8, color="black"
    )

plt.title("Unique Client IP Count by Province", fontsize=16)
plt.xlabel("Unique Client IP Count", fontsize=14)
plt.ylabel("Province", fontsize=12)

plt.yticks(x, unique_client_count.index, fontsize=8)
plt.grid(axis="x", linestyle="--", alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
test_counts_df = result_mapping_df.groupby(["city"]).size().compute()
test_counts_df = test_counts_df.reset_index(name="count")
test_counts_df = test_counts_df.sort_values(by="count", ascending=False)

### ข้อมูลเกี่ยวกับ Server ที่ใช้ทดสอบ

In [ ]:
server_info_df = result_mapping_df.groupby(["server_ip"]).size().compute()
server_info_df = server_info_df.reset_index(name="count")
server_info_df

In [ ]:
server_counts = server_info_df.merge(server_info, how="left", on="server_ip")
server_counts.dtypes

In [ ]:
print(len(server_info['country_code'].unique()))

In [ ]:
!pip install folium --quiet

In [ ]:
import folium
import branca.colormap as cm
import numpy as np

grouped_pd = server_counts.groupby(['latitude', 'longitude', 'country_code'])['count'].sum().reset_index()
center_lat = grouped_pd['latitude'].mean()
center_lon = grouped_pd['longitude'].mean()
grouped_pd = grouped_pd.dropna(subset=['count'])

log_counts = np.log1p(grouped_pd['count'])
vmin = log_counts.min()
vmax = log_counts.max()

colormap = cm.LinearColormap(
    colors=list(reversed(cm.linear.RdYlBu_09.colors)),
    vmin=vmin,
    vmax=vmax
)

colormap.caption = "Server Count"

m = folium.Map(location=[center_lat, center_lon], zoom_start=4)

for _, row in grouped_pd.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color=colormap(np.log1p(row['count'])),
        fill=True,
        fill_opacity=0.8,
        popup=folium.Popup(
            f"<b>Country:</b> {row['country_code']}<br>"
            f"<b>Count:</b> {row['count']:,}",
            max_width=300
        )
    ).add_to(m)

colormap.add_to(m)
m

In [ ]:
grouped_pd = server_counts.groupby(['server_ip', 'country_code'])['count'].sum().reset_index()
grouped_pd.sort_values(by='count', ascending=False, inplace=True)
grouped_pd.head(20)

### การกระจายตัวของประเภท ISP ของแต่ละจังหวัด

In [ ]:
grouped_counts = result_mapping_df.groupby(["city", "category"]).size().compute()
grouped_counts_df = grouped_counts.reset_index(name="count")
pivot_category_df = grouped_counts_df.pivot(index="city", columns="category", values="count").fillna(0)
pivot_category_df

In [ ]:
pivot_category = pivot_category_df.reset_index().set_index("city").T

pivot_clipped = pivot_category.clip(upper=100000)
pivot_log = np.log1p(pivot_clipped)

fig, ax = plt.subplots(figsize=(30, 12))

sns.heatmap(
    pivot_log,
    cmap="YlGnBu",
    annot=False,
    linewidths=0.25,
    ax=ax,
    cbar_kws={'label': 'Log(Count + 1)'}
)

ax.set_title("Heatmap of Category by Province", fontsize=18)
ax.set_xlabel("Province", fontsize=14)
ax.set_ylabel("Category", fontsize=14)

plt.xticks(rotation=90, ha='right', fontsize=12)
plt.yticks(fontsize=12)

ax.grid(which='major', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

### เวลาในการทดสอบ และความเร็วที่ได้ในแต่ละจังหวัด

#### Broadband

In [ ]:
province_hour_broadband_df = result_mapping_df.copy()[[
    'city', 'test_time', 'network_type', 'hour',
    'mean_throughput_mbps', 'type'
]].compute()

province_hour_broadband_df = remove_outliers_iqr(province_hour_broadband_df, ['city', 'hour', 'type', 'network_type'], 'mean_throughput_mbps')

province_hour_broadband_df = province_hour_broadband_df[
    province_hour_broadband_df["network_type"] == "broadband"
]

numeric_cols = province_hour_broadband_df.select_dtypes(include='number').columns

province_hour_broadband_df = province_hour_broadband_df.groupby(['city', 'hour', 'type'])[numeric_cols].mean()

province_hour_broadband_df.head()

In [ ]:
if "hour" in province_hour_broadband_df.columns:
    province_hour_broadband_df = province_hour_broadband_df.drop(columns=["hour"])

pivot_hour_broadband = province_hour_broadband_df.pivot_table(
    index="city",
    columns=["type", "hour"],
    values="mean_throughput_mbps",
    aggfunc="mean"
)

In [ ]:
pivot_hour_broadband.columns = [f"{t}_{h}" for t, h in pivot_hour_broadband.columns]

In [ ]:
pivot_hour_broadband.dtypes

In [ ]:
hour_d_broadband = pivot_hour_broadband[[col for col in pivot_hour_broadband.columns if col.startswith("download_")]]
hour_u_broadband = pivot_hour_broadband[[col for col in pivot_hour_broadband.columns if col.startswith("upload_")]]

hour_d_broadband.columns = [int(col.replace("download_", "")) for col in hour_d_broadband.columns]
hour_u_broadband.columns = [int(col.replace("upload_", "")) for col in hour_u_broadband.columns]
max_height = 20

plt.figure(figsize=(10, min(len(df) / 4, max_height)))
sns.heatmap(hour_d_broadband, cmap="RdYlBu_r", linewidths=0.5, robust=True)
plt.title("Mean Download Speed (Mbps) by Province and Hour (Broadband)", fontsize=12)
plt.xlabel("Hour")
plt.ylabel("Province")
plt.yticks(fontsize=8)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, min(len(df) / 4, max_height)))
sns.heatmap(hour_u_broadband, cmap="RdYlBu_r", linewidths=0.5, robust=True)
plt.title("Mean Upload Speed (Mbps) by Province and Hour (Broadband)", fontsize=12)
plt.xlabel("Hour")
plt.ylabel("Province")
plt.yticks(fontsize=8)
plt.tight_layout()
plt.show()

#### Cellular

In [ ]:
province_hour_mobile_df = result_mapping_df.copy()[[
    'city', 'test_time', 'network_type', 'hour',
    'mean_throughput_mbps', 'type'
]].compute()

province_hour_mobile_df = remove_outliers_iqr(province_hour_mobile_df, ['city', 'hour', 'type', 'network_type'], 'mean_throughput_mbps')

province_hour_mobile_df = province_hour_mobile_df[
    province_hour_mobile_df["network_type"] == "cellular"
]

numeric_cols = province_hour_mobile_df.select_dtypes(include='number').columns

province_hour_mobile_df = province_hour_mobile_df.groupby(['city', 'hour', 'type'])[numeric_cols].mean()

In [ ]:
if "hour" in province_hour_mobile_df.columns:
    province_hour_mobile_df = province_hour_mobile_df.drop(columns=["hour"])

pivot_hour_mobile = province_hour_mobile_df.pivot_table(
    index="city",
    columns=["type", "hour"],
    values="mean_throughput_mbps",
    aggfunc="mean"
)

In [ ]:
pivot_hour_mobile.columns = [f"{t}_{h}" for t, h in pivot_hour_mobile.columns]

In [ ]:
hour_d_mobile = pivot_hour_mobile[[col for col in pivot_hour_mobile.columns if col.startswith("download_")]]
hour_u_mobile = pivot_hour_mobile[[col for col in pivot_hour_mobile.columns if col.startswith("upload_")]]

hour_d_mobile.columns = [int(col.replace("download_", "")) for col in hour_d_mobile.columns]
hour_u_mobile.columns = [int(col.replace("upload_", "")) for col in hour_u_mobile.columns]
max_height = 20

plt.figure(figsize=(10, min(len(df) / 4, max_height)))
sns.heatmap(hour_d_mobile, cmap="RdYlBu_r", linewidths=0.5, robust=True)
plt.title("Mean Download Speed (Mbps) by Province and Hour (Cellular)", fontsize=12)
plt.xlabel("Hour")
plt.ylabel("Province")
plt.yticks(fontsize=8)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, min(len(df) / 4, max_height)))
sns.heatmap(hour_u_mobile, cmap="RdYlBu_r", linewidths=0.5, robust=True)
plt.title("Mean Upload Speed (Mbps) by Province and Hour (Cellular)", fontsize=12)
plt.xlabel("Hour")
plt.ylabel("Province")
plt.yticks(fontsize=8)
plt.tight_layout()
plt.show()

### แผนที่ประเทศไทยแสดงการเปลี่ยนแปลงของ Throughput

In [ ]:
tha_map = gpd.read_file("/content/THA_MAP.shx")

In [ ]:
provice_name = pd.read_csv("/content/Province_NAME.csv", index_col=None)
tha_map["NAME"] = provice_name["Province"]
tha_map.head()

#### แผนที่ Broadband

In [ ]:
province_map_b_df = result_mapping_df.copy()
province_map_b_df = province_map_b_df[province_map_b_df["network_type"] == "broadband"]
province_map_b_df = province_map_b_df.rename(columns={'city': 'province'})

In [ ]:
median_b_df = province_map_b_df.groupby(["province", "year", "month", "type"]) \
    .agg(
        median_throughput=("mean_throughput_mbps", "median"),
        test_count=("mean_throughput_mbps", "count")
    ).reset_index()

In [ ]:
median_b_df = median_b_df.dropna()
median_b_df = median_b_df[median_b_df["test_count"] >= 30]
median_b_df = median_b_df.compute()

In [ ]:
output_folder = "/content/exported_maps_broadband"
os.makedirs(output_folder, exist_ok=True)

years = median_b_df['year'].unique().tolist()
print(years)
vmin = median_b_df["median_throughput"].quantile(0.05)
vmax = median_b_df["median_throughput"].quantile(0.95)

for year in years:
    months = sorted(median_b_df[median_b_df['year'] == year]['month'].unique())
    print(months)
    for month in months:
        type_values = median_b_df[
            (median_b_df["year"] == year) & (median_b_df["month"] == month)
        ]["type"].unique()

        for net_type in type_values:
            filtered = median_b_df[
                (median_b_df["year"] == year) &
                (median_b_df["month"] == month) &
                (median_b_df["type"] == net_type)
            ]
            if filtered.empty:
                print(f"ไม่พบข้อมูลสำหรับ {month}/{year} - {net_type}")
                continue

            merged_gdf = tha_map.merge(filtered, left_on="NAME", right_on="province", how="left")
            simplified_gdf = merged_gdf.copy()
            simplified_gdf["geometry"] = simplified_gdf["geometry"].simplify(tolerance=0.05, preserve_topology=True)
            merged_geojson = json.loads(simplified_gdf.to_json())

            fig = px.choropleth(
                simplified_gdf,
                geojson=merged_geojson,
                locations="NAME",
                featureidkey="properties.NAME",
                color="median_throughput",
                color_continuous_scale="Turbo",
                range_color=(vmin, vmax),
                projection="mercator",
                title=f"Broadband Median Throughput (Mbps) - {month}/{year} ({net_type})",
            )
            fig.update_geos(fitbounds="locations", visible=False)
            fig.update_layout(width=1000, height=700, margin={"r":0,"t":50,"l":0,"b":0})
            safe_type = net_type.lower().replace(" ", "_")
            filename = f"{output_folder}/map_{year}_{month:02d}_{safe_type}.html"
            fig.write_html(filename, include_plotlyjs="inline")
            print(f"✅ Exported {filename}")

In [ ]:
output_folder = "/content/exported_maps_broadband_all"
os.makedirs(output_folder, exist_ok=True)

median_b_df["year_month_dt"] = pd.to_datetime(
    median_b_df["year"].astype(str) + "-" + median_b_df["month"].astype(str).str.zfill(2)
)
median_b_df["year_month"] = median_b_df["year_month_dt"].dt.strftime("%Y-%m")

vmin = median_b_df["median_throughput"].quantile(0.05)
vmax = median_b_df["median_throughput"].quantile(0.95)

network_types = median_b_df["type"].unique()

for net_type in network_types:
    filtered = median_b_df[median_b_df["type"] == net_type].copy()
    filtered = filtered.sort_values("year_month_dt")

    merged_gdf = tha_map.merge(filtered, left_on="NAME", right_on="province", how="left")
    merged_gdf["geometry"] = merged_gdf["geometry"].simplify(tolerance=0.05, preserve_topology=True)

    merged_gdf = merged_gdf.drop(columns=["year_month_dt"], errors="ignore")

    merged_geojson = json.loads(merged_gdf.to_json())

    fig = px.choropleth(
        merged_gdf,
        geojson=merged_geojson,
        locations="NAME",
        featureidkey="properties.NAME",
        color="median_throughput",
        animation_frame="year_month",
        color_continuous_scale="Turbo",
        range_color=(vmin, vmax),
        projection="mercator",
        title=f"Broadband Median Throughput (Mbps) ({net_type})"
    )
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(width=1000, height=700, margin={"r":0,"t":50,"l":0,"b":0})

    safe_type = net_type.lower().replace(" ", "_")
    filename = f"{output_folder}/map_all_months_{safe_type}_animated.html"
    fig.write_html(filename, include_plotlyjs="inline")
    print(f"✅ Exported animated map: {filename}")

In [ ]:
years = [2023, 2024, 2025]
months_by_year = {
    2023: [12],
    2024: list(range(1, 13)),
    2025: [1, 2, 3],
}
types_available = ['download', 'upload']
base_path = "/content/exported_maps_broadband"

html_widgets = {}

print("📦 Loading and building widgets...")
for y in years:
    for m in months_by_year[y]:
        for t in types_available:
            key = (y, m, t)
            path = f"{base_path}/map_{y}_{m:02d}_{t}.html"
            if os.path.exists(path):
                with open(path, "r", encoding="utf-8") as f:
                    html = f.read()
            else:
                html = f"<p style='color:red'>ไม่พบไฟล์: {path}</p>"

            html_widget = widgets.HTML(value=html, layout=widgets.Layout(display="none"))
            html_widgets[key] = html_widget

print("✅ All widgets loaded.")

year_dropdown = widgets.Dropdown(options=years, value=2024, description='Year:')
month_dropdown = widgets.Dropdown(options=months_by_year[2024], value=5, description='Month:')
type_dropdown = widgets.Dropdown(options=types_available, value='download', description='Type:')


def update_visibility():
    for w in html_widgets.values():
        w.layout.display = "none"
    y, m, t = year_dropdown.value, month_dropdown.value, type_dropdown.value
    key = (y, m, t)
    if key in html_widgets:
        html_widgets[key].layout.display = "block"

def on_year_change(change):
    y = change['new']
    month_dropdown.options = months_by_year[y]
    if month_dropdown.value not in month_dropdown.options:
        month_dropdown.value = month_dropdown.options[0]
    update_visibility()

year_dropdown.observe(on_year_change, names='value')
month_dropdown.observe(lambda change: update_visibility(), names='value')
type_dropdown.observe(lambda change: update_visibility(), names='value')

ui = widgets.HBox([year_dropdown, month_dropdown, type_dropdown])
all_maps_box = widgets.VBox(list(html_widgets.values()))
display(ui, all_maps_box)
update_visibility()

#### แผนที่ Cellular

In [ ]:
province_map_m_df = result_mapping_df.copy()
province_map_m_df = province_map_m_df[province_map_m_df["network_type"] == "cellular"]
province_map_m_df = province_map_m_df.rename(columns={'city': 'province'})

In [ ]:
median_m_df = province_map_m_df.groupby(["province", "year", "month", "type"]) \
    .agg(
        median_throughput=("mean_throughput_mbps", "median"),
        test_count=("mean_throughput_mbps", "count")
    ).reset_index()

In [ ]:
median_m_df = median_m_df.dropna()
median_m_df = median_m_df[median_m_df["test_count"] >= 30]
median_m_df = median_m_df.compute()

In [ ]:
output_folder = "/content/exported_maps_mobile"
os.makedirs(output_folder, exist_ok=True)

vmin = median_m_df["median_throughput"].min()
vmax = median_m_df["median_throughput"].max()

years = median_m_df['year'].unique().tolist()
for year in years:
    months = sorted(median_m_df[median_m_df['year'] == year]['month'].unique())
    for month in months:
        type_values =  median_m_df[
            (median_m_df["year"] == year) & (median_m_df["month"] == month)
        ]["type"].unique()

        for net_type in type_values:
            filtered = median_m_df[
                (median_m_df["year"] == year) &
                (median_m_df["month"] == month) &
                (median_m_df["type"] == net_type)
            ]
            if filtered.empty:
                print(f"ไม่พบข้อมูลสำหรับ {month}/{year} - {net_type}")
                continue

            merged_gdf = tha_map.merge(filtered, left_on="NAME", right_on="province", how="left")
            simplified_gdf = merged_gdf.copy()
            simplified_gdf["geometry"] = simplified_gdf["geometry"].simplify(tolerance=0.05, preserve_topology=True)
            merged_geojson = json.loads(simplified_gdf.to_json())

            fig = px.choropleth(
                simplified_gdf,
                geojson=merged_geojson,
                locations="NAME",
                featureidkey="properties.NAME",
                color="median_throughput",
                color_continuous_scale="Turbo",
                range_color=(vmin, vmax),
                projection="mercator",
                title=f"Cellular Mean Throughput (Mbps) - {month}/{year} ({net_type})"
            )
            fig.update_geos(fitbounds="locations", visible=False)
            fig.update_layout(width=1000, height=700, margin={"r":0,"t":50,"l":0,"b":0})

            safe_type = net_type.lower().replace(" ", "_")
            filename = f"{output_folder}/map_{year}_{month:02d}_{safe_type}.html"
            fig.write_html(filename)
            print(f"✅ Exported {filename}")

In [ ]:
output_folder = "/content/exported_maps_cellular_all"
os.makedirs(output_folder, exist_ok=True)

median_m_df["year_month"] = median_m_df.apply(lambda row: f"{row['year']}-{int(row['month']):02d}", axis=1)

vmin = median_m_df["median_throughput"].quantile(0.05)
vmax = median_m_df["median_throughput"].quantile(0.95)

network_types = median_b_df["type"].unique()

for net_type in network_types:
    filtered = median_m_df[median_m_df["type"] == net_type]
    if filtered.empty:
        print(f"ไม่พบข้อมูลสำหรับ {net_type}")
        continue

    merged_gdf = tha_map.merge(filtered, left_on="NAME", right_on="province", how="left")
    merged_gdf["geometry"] = merged_gdf["geometry"].simplify(tolerance=0.05, preserve_topology=True)
    merged_geojson = json.loads(merged_gdf.to_json())

    fig = px.choropleth(
        merged_gdf,
        geojson=merged_geojson,
        locations="NAME",
        featureidkey="properties.NAME",
        color="median_throughput",
        animation_frame="year_month",
        color_continuous_scale="Turbo",
        range_color=(vmin, vmax),
        projection="mercator",
        title=f"Cellular Median Throughput (Mbps) ({net_type})"
    )
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(width=1000, height=700, margin={"r":0,"t":50,"l":0,"b":0})

    safe_type = net_type.lower().replace(" ", "_")
    filename = f"{output_folder}/map_all_months_{safe_type}_animated.html"
    fig.write_html(filename, include_plotlyjs="inline")
    print(f"✅ Exported animated map: {filename}")

In [ ]:
output_folder = "/content/exported_maps_cellular_all"
os.makedirs(output_folder, exist_ok=True)

median_m_df["year_month_dt"] = pd.to_datetime(
    median_m_df["year"].astype(str) + "-" + median_m_df["month"].astype(str).str.zfill(2)
)
median_m_df["year_month"] = median_m_df["year_month_dt"].dt.strftime("%Y-%m")

vmin = median_m_df["median_throughput"].quantile(0.05)
vmax = median_m_df["median_throughput"].quantile(0.95)

network_types = median_b_df["type"].unique()

for net_type in network_types:
    filtered = median_m_df[median_m_df["type"] == net_type].copy()
    filtered = filtered.sort_values("year_month_dt")

    merged_gdf = tha_map.merge(filtered, left_on="NAME", right_on="province", how="left")
    merged_gdf["geometry"] = merged_gdf["geometry"].simplify(tolerance=0.05, preserve_topology=True)

    merged_gdf = merged_gdf.drop(columns=["year_month_dt"], errors="ignore")

    merged_geojson = json.loads(merged_gdf.to_json())

    fig = px.choropleth(
        merged_gdf,
        geojson=merged_geojson,
        locations="NAME",
        featureidkey="properties.NAME",
        color="median_throughput",
        animation_frame="year_month",
        color_continuous_scale="Turbo",
        range_color=(vmin, vmax),
        projection="mercator",
        title=f"Cellular Median Throughput (Mbps) ({net_type})"
    )
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(width=1000, height=700, margin={"r":0,"t":50,"l":0,"b":0})

    safe_type = net_type.lower().replace(" ", "_")
    filename = f"{output_folder}/map_all_months_{safe_type}_animated.html"
    fig.write_html(filename, include_plotlyjs="inline")
    print(f"✅ Exported animated map: {filename}")

In [ ]:
years = [2023, 2024, 2025]
months_by_year = {
    2023: [12],
    2024: list(range(1, 13)),
    2025: [1, 2, 3],
}
types_available = ['download', 'upload']
base_path = "/content/exported_maps_mobile"

html_widgets = {}

print("📦 Loading and building widgets...")
for y in years:
    for m in months_by_year[y]:
        for t in types_available:
            key = (y, m, t)
            path = f"{base_path}/map_{y}_{m:02d}_{t}.html"
            if os.path.exists(path):
                with open(path, "r", encoding="utf-8") as f:
                    html = f.read()
            else:
                html = f"<p style='color:red'>ไม่พบไฟล์: {path}</p>"

            html_widget = widgets.HTML(value=html, layout=widgets.Layout(display="none"))
            html_widgets[key] = html_widget

print("✅ All widgets loaded.")

year_dropdown = widgets.Dropdown(options=years, value=2024, description='Year:')
month_dropdown = widgets.Dropdown(options=months_by_year[2024], value=5, description='Month:')
type_dropdown = widgets.Dropdown(options=types_available, value='download', description='Type:')


def update_visibility():
    for w in html_widgets.values():
        w.layout.display = "none"
    y, m, t = year_dropdown.value, month_dropdown.value, type_dropdown.value
    key = (y, m, t)
    if key in html_widgets:
        html_widgets[key].layout.display = "block"

def on_year_change(change):
    y = change['new']
    month_dropdown.options = months_by_year[y]
    if month_dropdown.value not in month_dropdown.options:
        month_dropdown.value = month_dropdown.options[0]
    update_visibility()

year_dropdown.observe(on_year_change, names='value')
month_dropdown.observe(lambda change: update_visibility(), names='value')
type_dropdown.observe(lambda change: update_visibility(), names='value')

ui = widgets.HBox([year_dropdown, month_dropdown, type_dropdown])
all_maps_box = widgets.VBox(list(html_widgets.values()))
display(ui, all_maps_box)
update_visibility()

### RTT (Server ที่ใช้เยอะที่สุด)

In [ ]:
server_counts = result_mapping_df.groupby('server_ip').size().compute().sort_values(ascending=False)
top_server_ip = server_counts.idxmax()
top_server_count = server_counts.max()
print("Top server IP:", top_server_ip)
print("Number of tests:", top_server_count)

In [ ]:
rtt_df = result_mapping_df[result_mapping_df["server_ip"] == top_server_ip]
rtt_df = rtt_df.groupby(['city', 'type', 'network_type']).agg({
    'loss_rate': ['mean'],
    'min_rtt': ['mean'],
}).reset_index()

In [ ]:
rtt_df = rtt_df.dropna(subset=[('min_rtt', 'mean')])

In [ ]:
rtt_df.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in rtt_df.columns]

In [ ]:
rtt_df = rtt_df.compute()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import ScalarFormatter

def format_k(x):
    return f"{x/1000:.1f}k" if x >= 1000 else f"{x:.0f}"

fig, axes = plt.subplots(2, 2, figsize=(28, 16))

configs = [
    ("broadband", "download", axes[0, 0], "skyblue"),
    ("broadband", "upload", axes[0, 1], "salmon"),
    ("cellular", "download", axes[1, 0], "skyblue"),
    ("cellular", "upload", axes[1, 1], "salmon"),
]

for net_type, traffic_type, ax, color in configs:
    subset = rtt_df[
        (rtt_df["network_type"] == net_type) &
        (rtt_df["type"] == traffic_type)
    ].sort_values("min_rtt_mean")

    bars = sns.barplot(
        data=subset,
        x="city",
        y="min_rtt_mean",
        ax=ax,
        color=color
    )

    ax.set_yscale("log")

    formatter = ScalarFormatter(useMathText=True)
    formatter.set_powerlimits((0, 0))
    ax.yaxis.set_major_formatter(formatter)

    ax.set_title(
        f"Mean RTT by Province\n({net_type.title()} - {traffic_type.title()})",
        fontsize=18
    )
    ax.set_xlabel("Province", fontsize=14)
    ax.set_ylabel("Mean RTT (ms)", fontsize=14)
    ax.tick_params(axis='x', rotation=90, labelsize=10)
    ax.tick_params(axis='y', labelsize=12)

    for container in ax.containers:
      for bar in container:
          height = bar.get_height()
          if height > 0:
              ax.text(
                  bar.get_x() + bar.get_width() / 2,
                  height * 1.1,
                  f"{format_k(height)}",
                  ha='center',
                  va='bottom',
                  fontsize=8
              )

plt.tight_layout()
plt.show()

### Loss Rate

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import ScalarFormatter

fig, axes = plt.subplots(2, 2, figsize=(28, 16))

configs = [
    ("broadband", "download", axes[0, 0], "skyblue"),
    ("broadband", "upload", axes[0, 1], "salmon"),
    ("cellular", "download", axes[1, 0], "skyblue"),
    ("cellular", "upload", axes[1, 1], "salmon"),
]

for net_type, traffic_type, ax, color in configs:
    subset = rtt_df[
        (rtt_df["network_type"] == net_type) &
        (rtt_df["type"] == traffic_type)
    ].sort_values("loss_rate_mean")

    bars = sns.barplot(
        data=subset,
        x="city",
        y="loss_rate_mean",
        ax=ax,
        color=color
    )

    ax.set_title(
        f"Mean Loss Rate by Province\n({net_type.title()} - {traffic_type.title()})",
        fontsize=18
    )
    ax.set_xlabel("Province", fontsize=14)
    ax.set_ylabel("Mean Loss Rate", fontsize=14)
    ax.tick_params(axis='x', rotation=90, labelsize=10)
    ax.tick_params(axis='y', labelsize=12)

    for container in ax.containers:
      for bar in container:
          height = bar.get_height()
          if height > 0:
              ax.text(
                  bar.get_x() + bar.get_width() / 2,
                  height + 0.001,
                  f"{height:.2f}",
                  ha='center',
                  va='bottom',
                  fontsize=8,
                  rotation=45
              )

plt.tight_layout()
plt.show()

### ผู้ให้บริการอินเตอร์เน็ตบรอดแบนด์ (ทั่วไป)

In [ ]:
province_network_type_df = result_mapping_df.copy()

province_network_type_df = province_network_type_df.dropna(subset=["city"])
province_network_type_df = province_network_type_df[province_network_type_df["category"] == "Consumer Broadband"]

In [ ]:
province_network_type_df = province_network_type_df.groupby(['city', 'type', 'description']).agg(
    median_throughput=("mean_throughput_mbps", "median"),
    test_count=("mean_throughput_mbps", "count")
).dropna().reset_index()

In [ ]:
throughput_pivot = province_network_type_df.pivot_table(
    index="city",
    columns=["type", "description"],
    values="median_throughput",
)
throughput_pivot[throughput_pivot["city"] == "Mae Hong Son"]

In [ ]:
top_cities = throughput_pivot.xs("download", level="type", axis=1).mean(axis=1).nlargest(15).index
throughput_download = throughput_pivot.xs("download", level="type", axis=1).loc[top_cities]

fig, ax = plt.subplots(figsize=(18, 10))
sns.heatmap(throughput_download, cmap="Blues", annot=True, fmt=".1f", linewidths=3, ax=ax)
ax.set_title("Heatmap of Broadband Median Speed by Province and ISP (Download)", fontsize=16)
ax.set_xlabel("ISP", fontsize=14)
ax.set_ylabel("Province", fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

top_cities = throughput_pivot.xs("upload", level="type", axis=1).mean(axis=1).nlargest(15).index
throughput_upload = throughput_pivot.xs("upload", level="type", axis=1).loc[top_cities]

fig, ax = plt.subplots(figsize=(18, 10))
sns.heatmap(throughput_upload, cmap="Oranges", annot=True, fmt=".1f", linewidths=3, ax=ax)
ax.set_title("Heatmap of Broadband Median Speed by Province and ISP (Upload)", fontsize=16)
ax.set_xlabel("ISP", fontsize=14)
ax.set_ylabel("Province", fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
throughput_download = throughput_pivot.xs("download", level="type", axis=1)
throughput_upload = throughput_pivot.xs("upload", level="type", axis=1)

fig, ax = plt.subplots(figsize=(18, 30))
sns.heatmap(throughput_download, cmap="Blues", annot=True, fmt=".1f", linewidths=3, ax=ax)
ax.set_title("Heatmap of Broadband Median Speed by Province and ISP (Download)", fontsize=16)
ax.set_xlabel("ISP", fontsize=14)
ax.set_ylabel("Province", fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(18, 30))
sns.heatmap(throughput_upload, cmap="Oranges", annot=True, fmt=".1f", linewidths=3, ax=ax)
ax.set_title("Heatmap of Broadband Median Speed by Province and ISP (Upload)", fontsize=16)
ax.set_xlabel("ISP", fontsize=14)
ax.set_ylabel("Province", fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

### ผู้ให้บริการอินเตอร์เน็ตมือถือ

In [ ]:
province_network_type_df = result_mapping_df.copy()

province_network_type_df = province_network_type_df.dropna(subset=["city"])
province_network_type_df = province_network_type_df[province_network_type_df["category"] == "Mobile"]

In [ ]:
province_network_type_df = province_network_type_df.groupby(['city', 'type', 'description']).agg(
    median_throughput=("mean_throughput_mbps", "median"),
    test_count=("mean_throughput_mbps", "count")
).dropna().compute().reset_index()
province_network_type_df.head()

In [ ]:
province_network_type_df = province_network_type_df[province_network_type_df["test_count"] >= 30]

In [ ]:
throughput_pivot = province_network_type_df.pivot_table(
    index="city",
    columns=["type", "description"],
    values="median_throughput",
)
throughput_pivot

In [ ]:
top_cities = throughput_pivot.xs("download", level="type", axis=1).mean(axis=1).nlargest(15).index
throughput_download = throughput_pivot.xs("download", level="type", axis=1).loc[top_cities]

fig, ax = plt.subplots(figsize=(18, 10))
sns.heatmap(throughput_download, cmap="Blues", annot=True, fmt=".1f", linewidths=3, ax=ax)
ax.set_title("Heatmap of Cellular Median Speed by Province and ISP (Download)", fontsize=16)
ax.set_xlabel("ISP", fontsize=14)
ax.set_ylabel("Province", fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

top_cities = throughput_pivot.xs("upload", level="type", axis=1).mean(axis=1).nlargest(15).index
throughput_upload = throughput_pivot.xs("upload", level="type", axis=1).loc[top_cities]

fig, ax = plt.subplots(figsize=(18, 10))
sns.heatmap(throughput_upload, cmap="Oranges", annot=True, fmt=".1f", linewidths=3, ax=ax)
ax.set_title("Heatmap of Upload Median Speed by Province and ISP (Upload)", fontsize=16)
ax.set_xlabel("ISP", fontsize=14)
ax.set_ylabel("Province", fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
throughput_download = throughput_pivot.xs("download", level="type", axis=1)
throughput_upload = throughput_pivot.xs("upload", level="type", axis=1)

fig, ax = plt.subplots(figsize=(18, 20))
sns.heatmap(throughput_download, cmap="Blues", annot=True, fmt=".1f", linewidths=3, ax=ax)
ax.set_title("Heatmap of Cellular Median Speed by Province and ISP (Download)", fontsize=16)
ax.set_xlabel("ISP", fontsize=14)
ax.set_ylabel("Province", fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(18, 20))
sns.heatmap(throughput_upload, cmap="Oranges", annot=True, fmt=".1f", linewidths=3, ax=ax)
ax.set_title("Heatmap of Cellular Median Speed by Province and ISP (Upload)", fontsize=16)
ax.set_xlabel("ISP", fontsize=14)
ax.set_ylabel("Province", fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()